### Step1: feature extraction 

In [33]:
from data_processor import train_loader, test_loader, unlabeled_loader
import torch
import torch.nn as nn
from torchvision import models
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading

In [34]:
model = models.alexnet(pretrained=True)

In [35]:
new_classifier = nn.Sequential(*list(model.classifier.children())[:-4])
model.classifier = new_classifier

In [36]:
with torch.no_grad(): 
    batch = next(iter(train_loader))
    images, labels = batch
    all_out = model(images)
    all_labels = labels 

    for batch in train_loader:
        images, labels = batch 
        all_out = torch.cat((all_out, model(images)), 0)
        all_labels = torch.cat((all_labels, labels), 0)

    for batch in unlabeled_loader:
        images, labels = batch
        all_out = torch.cat((all_out, model(images)), 0)
        all_labels = torch.cat((all_labels, labels+-1), 0)

In [37]:
print(all_out.shape)
print(all_labels.shape)

torch.Size([14873, 4096])
torch.Size([14873])


In [50]:
test_images, test_labels = next(iter(test_loader))
test_features = model(test_images)

with torch.no_grad(): 
    for images, labels in test_loader:
        test_features = torch.cat((test_features, model(images)), 0)
        test_labels = torch.cat((test_labels, labels), 0)
        

In [51]:
print(test_features.shape)
print(test_labels.shape)

torch.Size([1464, 4096])
torch.Size([1464])


In [57]:
label_prop_model = LabelSpreading(max_iter = 1000, kernel = 'rbf', alpha = 0.01)

label_prop_model.fit(all_out.numpy(), all_labels.numpy())

/usr/lib/python3/dist-packages/sklearn/semi_supervised/label_propagation.py:297: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


LabelSpreading(alpha=0.01, gamma=20, kernel='rbf', max_iter=1000, n_jobs=None,
        n_neighbors=7, tol=0.001)

In [58]:
label_prop_model.score(test_features.numpy(), test_labels.numpy())

0.1987704918032787